# Data preparation

## 0. Intro

In this notebook we process all the data for the 4 base models:
1. Data - Filters (theta, alpha, beta) - Window 64 data ((14x3x64)+1 = 2688 features) for SVM (RBF and Linear)
2. Data - Filters (theta, alpha, beta) - Window 128 data ((14x3x128)+1 = 5377 features) for SVM (RBF and Linear)
3. Data - Filters (theta, alpha, beta) - Window 64 data (14x3x64 = 2688 features) - PSD signal properties (14x3x6 = 252 features) for SVM (RBF and Linear)
4. Data - Filters (theta, alpha, beta) - Window 64 data (14x3x64 = 2688 features) - MaxMinScaler (to transform all from 0 to 1) - PCA (data reduction to half ((2688/2)+1 = 1345 features)) for SVM (RBF and Linear)

### 0.1 Import all the packages

In [1]:
import numpy as np
import scipy.signal as signal
from sklearn.decomposition import PCA

## 1. Data - Filters (theta, alpha, beta) - Window 64 data ((14x3x64)+1 = 2688 features) for SVM (RBF and Linear)

### 1.1. This function is for a and b parameters for the filters.

In [2]:
def param(N, Wn, fs):
    b, a = signal.butter(N, Wn, btype='bandpass', analog=False, output='ba', fs=fs)
    return a, b

### 1.2 This function is to filter all the signals in the 3 bands (Theta, Alpha and Theta) 

In [3]:
def filt(df, a_th, b_th, a_al, b_al, a_be, b_be, path, CI, tp, num):
    x_th = np.zeros(df.shape)
    x_al = np.zeros(df.shape)
    x_be = np.zeros(df.shape)
    y = df[:,14]
    for i in range(df.shape[1]):
        x_th[:, i] = signal.filtfilt(b_th, a_th, df[:, i])
        x_al[:, i] = signal.filtfilt(b_al, a_al, df[:, i])
        x_be[:, i] = signal.filtfilt(b_be, a_be, df[:, i])
    x_th[:,14] = y
    x_al[:,14] = y
    x_be[:,14] = y
    return x_th, x_al, x_be

### 1.3 Here we order the filtered signal acording the 64 data windowing making the 2687 features and adding the preview class, in total 2689 features. This function can do the windowing of the value of `win`. 

In [4]:
def dat_prep_win(x_th, x_al, x_be, df, base_l1, base_l2, win, jump, fs, tp, num, path, CI):
    if df[0, 14] == 5:
        base = int(base_l2*fs)
    else:
        base = int(base_l1*fs)

    shape = int(df.shape[0]-(base*2))

    X_shape1 = (14*3*win)+2
    X_shape0 = int((shape-win)/jump)+1
    X = np.empty((X_shape0, X_shape1))
    jump_to = 0
    for i in range(X_shape0):
        p = 0 
        for j in range(win):
            to_p = p
            p += 1
            X[i, (14*to_p):(14*p)] = x_th[base+jump_to+j, :14]
            to_p = p
            p += 1
            X[i, (14*to_p):(14*p)] = x_al[base+jump_to+j, :14]
            to_p = p
            p += 1
            X[i, (14*to_p):(14*p)] = x_be[base+jump_to+j, :14]
        X[i, X_shape1-2] = df[base+jump_to, 14]
        X[i, X_shape1-1] = df[base+jump_to, 14]
        jump_to += jump
    return X

### 1.5 Here we join the classes 1 and 2 in 1, the class 3 and 4 in 2 and the class 5 in 3, to have only 3 classes.

In [5]:
def five_to_three_class(X, win):
    y_new = np.empty((X.shape[0],2))
    y_new = X[:, (win*3*14):]
    y_new = np.where(y_new == 2, int(1), y_new)   #La clase 2 lo cambiamos a 1
    y_new = np.where(y_new == 4, int(3), y_new)   #La clase 4 lo cambiamos a 3
    y_new = np.where(y_new == 3, int(2), y_new)   #La clase 3 lo cambiamos a 2
    y_new = np.where(y_new == 5, int(3), y_new)   #La clase 5 lo cambiamos a 3

    X[:, (win*3*14):] = y_new
    return X

### 1.6 This function run all the process.

In [6]:
def main_func(win, base_l1, base_l2, jump, fs, N, path, CI):
    X_5C = np.empty((0,(win*3*14) + 2))
    X = np.empty((0,(win*3*14) + 2))
    a_th, b_th = param(N=N, Wn=[4, 7], fs=fs)
    a_al, b_al = param(N=N, Wn=[8, 12], fs=fs)
    a_be, b_be = param(N=N, Wn=[12, 30], fs=fs)
    for i in range(1, 9):
        for j in range(1, 8):
            df = np.genfromtxt(str(path)+'/'+str(CI)+'/data_clean_per_session/'+'Type_'+str(i)+'_'+str(j)+'.csv',dtype=float,delimiter=',',skip_header=1)
            x_th, x_al, x_be = filt(df=df, a_th=a_th, b_th=b_th, a_al=a_al, b_al=b_al, a_be=a_be, b_be=b_be, path=path, CI=CI, tp=i, num=j)
            x_append = dat_prep_win(x_th=x_th, x_al=x_al, x_be=x_be, df=df, base_l1=base_l1, base_l2=base_l2, win=win, jump=jump, fs=fs, tp=j, num=i, path=path, CI=CI)
            X_5C = np.append(X_5C, x_append, axis=0)
    X_3C = five_to_three_class(X=X_5C, win=win)
    print('Data_prep_'+str(win)+'_'+str(CI)+': Done')
    return X_3C

### 1.7 Here we define all the important values.

In [7]:
win = 64
base_l1 = 5
base_l2 = 2.5
jump = 16
fs = 256
N = 6
path = 'D:/Github/eeg.fem/public/data/Musical'
CI = 5956733

### 1.8 Start to make the data processing.

In [8]:
X_64 = main_func(win, base_l1, base_l2, jump, fs, N, path, CI)
np.savetxt(str(path)+'/'+str(CI)+'/data_for_train/ALL_3C_'+str(win)+'.csv', X_64, delimiter=',')

Data_prep_64_5956733: Done


## 2. Data - Filters (theta, alpha, beta) - Window 128 data ((14x3x128)+1 = 5377 features) for SVM (RBF and Linear)
For this data preparation we will use all the functions we saw before, so we have to set up all the variables. We have do define the steps *1.7* and run *1.8*.

### 2.1 Declaring variables

In [9]:
win = 128
base_l1 = 5
base_l2 = 2.5
jump = 16
fs = 256
N = 6
path = 'D:/Github/eeg.fem/public/data/Musical'
CI = 5956733

### 2.2 Running all the preprossesing data.

In [10]:
X_128 = main_func(win, base_l1, base_l2, jump, fs, N, path, CI)
np.savetxt(str(path)+'/'+str(CI)+'/data_for_train/ALL_3C_'+str(win)+'.csv', X_128, delimiter=',')

Data_prep_128_5956733: Done


## 3. Data - Filters (theta, alpha, beta) - Window 64 data (14x3x64 = 2688 features) - PSD signal properties (14x3x6 = 252 features) for SVM (RBF and Linear)

### 3.1 For this data pre we are going to start from the output of the first model. So we going to declare the variables like the first data pre.

In [13]:
win = 64
base_l1 = 5
base_l2 = 2.5
jump = 16
fs = 256
N = 6
path = 'D:/Github/eeg.fem/public/data/Musical'
CI = 5956733

### 3.2 We are going to run the step *1.8*.

In [14]:
X_64 = main_func(win, base_l1, base_l2, jump, fs, N, path, CI)

Data_prep_64_5956733: Done


### 3.3 From here we are going to create the function to preform the Power Spectral Density.

In [10]:
def psd_X(X, win, path, CI, fs):
    #Get m,n data.shape
    m, n = X.shape
    #Create the new_data matrix
    new_data = np.empty((X.shape[0],((6*14*3)+2)))
    x = np.empty((1, ((6*14*3)+2)))
    #Recorremos de i -> m
    for i in range(m):
    #Recorremos de j -> 42 recoriendo cada electrodo de cada filtro (14*3)
        l = 0
        for j in range(42):
            d_a = j
    #Recorremos la cantidad de win k -> win que es el window
            for k in range(win):
    #Ponemos en los indices x[0, k] = X[i, d_a]
                x[0, k] = X[i, d_a]
                d_a += 42
    #Sacamos las propiedades cada señal
            new_data[i, l] = np.amax(x) #Maximo valor en x
            new_data[i, l+1] = np.amin(x)   #Minimo valor en x
            new_data[i, l+2] = np.amax(x)-np.amin(x)    #Diferencia entre max y min
            new_data[i, l+3] = np.mean(x) #Valor promedio
            new_data[i, l+4] = np.sum(np.power(x, 2)/((win*1000)/fs))  #Power spectral density in ms
            new_data[i, l+5] = np.sum(np.power(x, 2))   #Energy spectral density
            l = l+6
    new_data[:, -2:] = X[:, -2:] #Ponemos old_y y y en los nuevos datos.
    print('PSD_Done')
    return new_data

### 3.4 Now we run the PSD function and save it.

In [15]:
X_psd = psd_X(X=X_64, win=win, path=path, CI=CI, fs=fs)
np.savetxt(str(path)+'/'+str(CI)+'/data_for_train/ALL_PSD_'+str(win)+'.csv', X_psd, delimiter=',')

PSD_Done


## 4. Data - Filters (theta, alpha, beta) - Window 64 data (14x3x64 = 2688 features) - MaxMinScaler (to transform all from 0 to 1) - PCA (data reduction to half ((2688/2)+1 = 1345 features)) for SVM (RBF and Linear)

### 4.1 In this function we determine the maximun and minimun values of each signal, this function return one matrix of (14, 6), in the raws we have the 14th electrodes and in the columns we have the max and min of each band of the electrode (theta, alpha and beta). 

In [6]:
def param_max_min(fil_max_min, i, j, x_th, x_al, x_be):
    if i == 1 and j == 1:
        for a in range(14):
            fil_max_min[a,0] = np.amax(x_th[1:,a])
            fil_max_min[a,1] = np.amin(x_th[1:,a])
            fil_max_min[a,2] = np.amax(x_al[1:,a])
            fil_max_min[a,3] = np.amin(x_al[1:,a])
            fil_max_min[a,4] = np.amax(x_be[1:,a])
            fil_max_min[a,5] = np.amin(x_be[1:,a])
    else:
        for a in range(14):
            if fil_max_min[a, 0] < np.amax(x_th[1:, a]):
                fil_max_min[a, 0] = np.amax(x_th[1:, a])
            if fil_max_min[a, 1] > np.amin(x_th[1:, a]):
                fil_max_min[a, 1] = np.amin(x_th[1:, a])
            if fil_max_min[a, 2] < np.amax(x_al[1:, a]):
                fil_max_min[a, 2] = np.amax(x_al[1:, a])
            if fil_max_min[a, 3] > np.amin(x_al[1:, a]):
                fil_max_min[a, 3] = np.amin(x_al[1:, a])
            if fil_max_min[a, 4] < np.amax(x_be[1:, a]):
                fil_max_min[a, 4] = np.amax(x_be[1:, a])
            if fil_max_min[a, 5] > np.amin(x_be[1:, a]):
                fil_max_min[a, 5] = np.amin(x_be[1:, a])
    return fil_max_min

### 4.2 This function is the same as the `main_func` but this function adds the `fil_max_min` matrix, to determine this we use the `para_max_min` function, and returns `X_3C` and `fil_max_min` matrices.

In [7]:
def main_func_pca(win, base_l1, base_l2, jump, fs, N, path, CI):
    X_5C = np.empty((0,(win*3*14) + 2))
    X = np.empty((0,(win*3*14) + 2))
    fil_max_min = np.empty((14, 6))
    a_th, b_th = param(N=N, Wn=[4, 7], fs=fs)
    a_al, b_al = param(N=N, Wn=[8, 12], fs=fs)
    a_be, b_be = param(N=N, Wn=[12, 30], fs=fs)
    for i in range(1, 9):
        for j in range(1, 8):
            df = np.genfromtxt(str(path)+'/'+str(CI)+'/data_clean_per_session/'+'Type_'+str(i)+'_'+str(j)+'.csv',dtype=float,delimiter=',',skip_header=1)
            x_th, x_al, x_be = filt(df=df, a_th=a_th, b_th=b_th, a_al=a_al, b_al=b_al, a_be=a_be, b_be=b_be, path=path, CI=CI, tp=i, num=j)
            fil_max_min = param_max_min(fil_max_min, i, j, x_th, x_al, x_be)
            x_append = dat_prep_win(x_th=x_th, x_al=x_al, x_be=x_be, df=df, base_l1=base_l1, base_l2=base_l2, win=win, jump=jump, fs=fs, tp=j, num=i, path=path, CI=CI)
            X_5C = np.append(X_5C, x_append, axis=0)
    X_3C = five_to_three_class(X=X_5C, win=win)
    print('Data_prep_'+str(win)+'_'+str(CI)+': Done')
    return X_3C, fil_max_min

### 4.3 This functio scaled the data acording to the given `fil_max_min` matrix and returns the X_scaled, the feature `y_old` and `y` remains the same.

In [8]:
def scaled(X, fil_max_min, win):
    X_scaled = np.empty((X.shape))
    idx = np.empty((42,64))
    for i in range (42):
        w = i
        for j in range(win):
            idx[i,j] = w
            w += 42
    max_min = 0
    k = 0
    for i in range (3):
        for j in range(14):
            idx_list = idx[k,:].astype(int)
            X_scaled[:, idx_list] = (X[:, idx_list]-fil_max_min[j,max_min+1])/(fil_max_min[j,max_min]-fil_max_min[j,max_min+1])
            k += 1
        max_min += 2
    X_scaled[:,-2:] = X[:,-2:]
    return X_scaled

### 4.4 In this function we apply PCA, the `y_old` feature doesnt enter to the PCA reduction.

In [9]:
def pca_X(X_scaled, n_comp, svd_solver, random_state):
    pca = PCA(n_components=n_comp, svd_solver=svd_solver, random_state=random_state)
    pca.fit(X_scaled[:,:-2])
    X_pca_i = pca.transform(X_scaled[:,:-2])
    X_pca = np.insert(X_pca_i, X_pca_i.shape[1], X_scaled[:, -2:].T, axis=1)
    print(X_pca_i.shape)
    print(X_pca.shape)
    return X_pca

### 4.3 We start the PCA function with the MaxMin scaler and next the PCA.

In [10]:
def main_pca(win, base_l1, base_l2, jump, fs, N, path, CI, n_comp, svd_solver, random_state):
    X, fil_max_min = main_func_pca(win, base_l1, base_l2, jump, fs, N, path, CI)
    #Escalamos las funciones de 0 a 1
    X_scaled = scaled(X, fil_max_min, win)
    #Hacemos PCA
    X_pca = pca_X(X_scaled, n_comp, svd_solver, random_state)
    return X_pca, fil_max_min, X_scaled

### 4.4 We set up all the variables.

In [11]:
win = 64
base_l1 = 5
base_l2 = 2.5
jump = 16
fs = 256
N = 6
path = 'D:/Github/eeg.fem/public/data/Musical'
CI = 5956733
n_comp = 10 #Number of components to be divided
svd_solver = 'randomized' #PCA solver
random_state = 21 #Random state of PCA solver

### 4.5 We run all the functions and save `X_pca` and the matrix `fil_max_min` were have all the max and min values of each channel of each filter.

In [12]:
X_pca, fil_max_min, X_scaled = main_pca(win, base_l1, base_l2, jump, fs, N, path, CI, n_comp, svd_solver, random_state)
np.savetxt(str(path)+'/'+str(CI)+'/data_for_train/ALL_PCA_'+str(win)+'.csv', X_pca, delimiter=',')
np.savetxt(str(path)+'/'+str(CI)+'/data_for_train/ALL_3C_'+str(win)+'_scaled.csv', X_scaled[:n_comp,:-2], delimiter=',')
np.savetxt(str(path)+'/'+str(CI)+'/ML_models/fil_max_min.csv', fil_max_min, delimiter=',')

Data_prep_64_5956733: Done
(6728, 10)
(6728, 12)
